In [1]:
import folium
from folium import plugins
import numpy as np
import pandas as pd
import geojson
import geopandas as gpd

data_directory = '/home/jan/Uni/DS-Project/data/Maps/'

In [4]:
# load germany maps & polygons
bund = gpd.read_file(data_directory + 'Bundeskarte.geojson').to_crs(3857)
land = gpd.read_file(data_directory + 'Laenderkarte.geojson').to_crs(3857)
kreis = gpd.read_file(data_directory + 'Kreiskarte.geojson').to_crs(3857)
polygon = gpd.read_file(data_directory + 'brandenburg_polygons.geojson').set_crs(25833, allow_override = True).to_crs(4326)

create offset to show legal distance to streets

In [60]:
polygon_offset = polygon.copy().to_crs(3043)
polygon_offset['geometry'] = polygon_offset.geometry.buffer(-12, single_sided = True)
polygon_offset['area'] = polygon_offset['geometry'].area / 10**3
polygon_offset['geometry'] = polygon_offset.geometry.to_crs(4326)

create general map without any information

In [80]:
Germany = folium.Map(location=[51.2, 10.5],
                 min_lon=5,
                 max_lon=15.7,
                 min_lat=55.5,
                 max_lat=46.5,
                 min_zoom=6,
                 zoom_start=6,
                 max_bounds=True,
                 control_scale=True,
                 tiles = "OpenStreetMap")
# different map layers
folium.TileLayer('openstreetmap').add_to(Germany)
folium.TileLayer('stamenterrain').add_to(Germany)
folium.LayerControl().add_to(Germany)
# implement extras
minimap = plugins.MiniMap(toggle_display = True)
Germany.add_child(minimap)
plugins.Fullscreen(position = 'topright').add_to(Germany)
plugins.ScrollZoomToggler().add_to(Germany)

fill map with data from geojson files

In [78]:
# define styles for different maps
    # grey fill with grey lines
style_kreis = {'fillColor': '#bfbfbf', 'color': '#727272'}
    # yellow fill with black lines
style_ploygon = {'fillColor': '#ffff6f', 'color': '#191903'}

folium.GeoJson(kreis,
               name = 'Kreisgrenzen',
               style_function = lambda x:style_kreis).add_to(Germany)
folium.GeoJson(polygon_offset["geometry"],
               name = 'Polygon Brandenburg',
               style_function = lambda x:style_ploygon
               ).add_to(Germany)
import dash_leaflet as dl
Germany.save('BB_ploygon.html')

In [81]:
Germany

In [70]:
polygon_offset.head(n = 5)

,group,access,area,bicycle,bicycle_road,bridge,busway,cycleway,foot,footway,...,width,id,timestamp,version,tags,osm_type,changeset,link_type,link_id,geometry
0,[ 0 1 2 727 765 766 1139 1140 1718 ...,None,24.220559,None,None,yes,None,None,None,None,...,None,4040480,1609519420,32,"{'destination': 'Hamburg;Schwerin', 'destinati...",way,None,ML,BB_ML_0000,"POLYGON ((12.45351 53.13539, 12.45367 53.13542..."
1,[ 0 1 2 727 765 766 1139 1140 1718 ...,None,38.985092,None,None,yes,None,None,None,None,...,None,4040480,1609519420,32,"{'destination': 'Hamburg;Schwerin', 'destinati...",way,None,ML,BB_ML_0000,"POLYGON ((12.45251 53.13514, 12.45261 53.13514..."
2,[ 0 1 2 727 765 766 1139 1140 1718 ...,None,254.824868,None,None,yes,None,None,None,None,...,None,4040480,1609519420,32,"{'destination': 'Hamburg;Schwerin', 'destinati...",way,None,ML,BB_ML_0000,"POLYGON ((12.45827 53.13488, 12.45842 53.13487..."
3,[ 0 1 2 727 765 766 1139 1140 1718 ...,None,34.885383,None,None,yes,None,None,None,None,...,None,4040480,1609519420,32,"{'destination': 'Hamburg;Schwerin', 'destinati...",way,None,ML,BB_ML_0000,"POLYGON ((12.46479 53.12816, 12.46461 53.12799..."
4,[ 0 1 2 727 765 766 1139 1140 1718 ...,None,0.000000,None,None,yes,None,None,None,None,...,None,4040480,1609519420,32,"{'destination': 'Hamburg;Schwerin', 'destinati...",way,None,ML,BB_ML_0000,POLYGON EMPTY


# test area

In [59]:
# change to meters
    # epsg: 6933(area), 3043(other area)
test = polygon_offset.to_crs(3043)
# create buffered polygons
test['geometry'] = test.geometry.buffer(-12, single_sided=True)
# calculate are in polygon
test['area'] = test['geometry'].area / 10**3
#test = test.to_crs(4326)

print(test.head(n = 5))
print(test.area.max())

                                               group access      area bicycle  \
0  [   0    1    2  727  765  766 1139 1140 1718 ...   None  0.015176    None   
1  [   0    1    2  727  765  766 1139 1140 1718 ...   None  0.025213    None   
2  [   0    1    2  727  765  766 1139 1140 1718 ...   None  0.230517    None   
3  [   0    1    2  727  765  766 1139 1140 1718 ...   None  0.022456    None   
4  [   0    1    2  727  765  766 1139 1140 1718 ...   None  0.000000    None   

  bicycle_road bridge busway cycleway  foot footway  ... width       id  \
0         None    yes   None     None  None    None  ...  None  4040480   
1         None    yes   None     None  None    None  ...  None  4040480   
2         None    yes   None     None  None    None  ...  None  4040480   
3         None    yes   None     None  None    None  ...  None  4040480   
4         None    yes   None     None  None    None  ...  None  4040480   

    timestamp version                                         

In [41]:
print(test.area[0])

2.0193867020718804e-06


/tmp/ipykernel_7502/2835498732.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(test.area[0])
